<a href="https://colab.research.google.com/github/matiasvallejosdev/my-face-recognition/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
 
from time import time #importamos la función time para capturar tiempos

In [5]:
from google.colab.patches import cv2_imshow

## Preprocesing multiples images
Paper reference:
[Image Pre-processing Using OpenCV Library on MORPH-II Face Database](https://uncw.edu/math/reu/documents/image-pre-processing.pdf)

## 1. Preparing alghoritm (Helpers)

1. Initialize in memory classifiers
2. Set parameters for cascade classifiers
3. Get data images_faces from drive

### Step 1.1

In [6]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Prepare data
faces_cascade = '/content/drive/My Drive/Machine Learning/haarcascade_frontalface_alt2.xml'
eyes_cascade = '/content/drive/My Drive/Machine Learning/haarcascade_eye_tree_eyeglasses.xml'

In [8]:
faces_cascade = cv2.CascadeClassifier(faces_cascade)
eyes_cascade = cv2.CascadeClassifier(eyes_cascade)

print("Load faces and eyes haarcascade")

Load faces and eyes haarcascade


### Step 1.2

In [40]:
## Faces
faceScale = 1.1
faceMinNeighborgs = 5
## Eyes
eyeScale = 1.1
eyesMinNeighborgs = 5

print("Face scale is:", faceScale)
print("Face minimal neighborgs is:", faceMinNeighborgs)
print("Face flags is:", faceFlags)

print("\nEye scale is:", eyeScale)
print("Eye minimal neighborgs is:", eyesMinNeighborgs)
print("Eye flags is:", eyesFlags)

Face scale is: 1.1
Face minimal neighborgs is: 5
Face flags is: 1

Eye scale is: 1.1
Eye minimal neighborgs is: 5
Eye flags is: 1


### Step 1.3

In [10]:
!ls "/content/drive/My Drive/Machine Learning/Dataset"

Me


In [11]:
# Get data and preparing
url_datapath = '/content/drive/My Drive/Machine Learning/Dataset/Me'
img_names= os.listdir(url_datapath)

print("Preparing data..")
print("Datapath from drive is:", url_datapath)
print("Total images collected:", len(img_names))

Preparing data..
Datapath from drive is: /content/drive/My Drive/Machine Learning/Dataset/Me
Total images collected: 3899



## 2. Preparing data for training a deep neural network

1. Define helpers
2. Construct Algoritm
  - Convert to grayscale
  - Face detection
  - Eye detection
  - Image Rotation
  - Face and Eye Re-detection
  - Cropping and Scaling
  - Save images
3. Manual Preprocesing Undetactable Images


### Step 2.1

In [32]:
def GetCropFace(imgGray, scale, minNeig):
    grayCrop = []
    faceDetection = faces_cascade.detectMultiScale(imgGray, scale, minNeig)
    print('Can detect(face): ', len(faceDetection))
    
    if len(faceDetection) == 1:
      (x, y, w, h) = faceDetection[0]
      grayCrop.append(imgGray[y:y+h, x:x+w])

      cv2.rectangle(imgGray, (x, y), (x+w, y+h), (0, 255, 0), 1);
      ShowImage(imgGray[y:y+h, x:x+w])

    return faceDetection, grayCrop

# Saving data
def SaveImage(img, total_facedetection, num):
    datapath_saving = '/content/drive/My Drive/Machine Learning/Preprocessing'

    if total_facedetection == 1:
        # Process finished correctly    
        datapath_saving = datapath_saving + '/Correct/' + 'out_' +str(num)+ '.jpg'
        print('Saving on: ', datapath_saving)
    elif total_facedetection > 1:
        # Multiples faces on image
        datapath_saving = datapath_saving + '/Multiple/' + 'mult_' +str(num)+ '.jpg'
        print('Saving on: ', datapath_saving)
    else:
        # Face undetacteble
        datapath_saving = datapath_saving + '/Undetect/' + 'undetect_' +str(num)+ '.jpg'
        print('Saving on: ', datapath_saving)

    SaveOpenCvImage(img, datapath_saving)

def SaveOpenCvImage(img, path):
    try:
        cv2.imwrite(path, img)
    except Exception as e:
        print('Exception: ', e)

# Show images
def ShowImage(img):    
    fig= plt.figure(figsize=(5, 5))
    rows = 1

    fig.add_subplot(rows, 1, 1)
    plt.imshow(img[:,:,::-1])
    plt.show()

In [14]:
def GetCropEyes(imgGray, faceDetection):
    faceGray_crop = []
    eyesDetection = []
    result = False

    eyeDetection = eyes_cascade.detectMultiScale(faceGray, eyeScale, eyesMinNeighborgs) # Eyes detector      
    print('Eyes detection: ', len(eyeDetection))

    if len(eyeDetection) != 2 or len(eyeDetection) < 2:
        return result, eyeDetection, faceGray_crop

    for (ex, ey ,ew, eh) in eyeDetection: 
        result = True
        cv2.rectangle(faceGray, (ex,ey), (ex+ew, ey+eh), (0, 255, 0), 4)
        faceGray_crop.append(faceGray) # Facegray
        
    return result, eyeDetection, faceGray_crop

In [13]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

### Step 1.1

In [65]:
import json
from google.colab import files

In [ ]:
min_image = 2000
max_image = len(img_names) - 1

widht_image = 32
height_image = 32

fail = 0
correct = 0
incorrect = 0

tiempo_inicial = time()

index = 2000

for imgname in img_names[min_image : max_image]:   
    index = index + 1
    
    img_path = os.path.join(url_datapath, imgname)
    img = cv2.imread(img_path)

    print('\nRead Image: ', imgname)
    try:
        # 1. Convert to gray scale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 2. Face detection
        faceDetection = faces_cascade.detectMultiScale(gray, faceScale, faceMinNeighborgs)
        print('Can detect(face): ', len(faceDetection))


        if len(faceDetection) == 1:
          (x, y, w, h) = faceDetection[0]

          # 3. Eye detection
          eyesDetection = eyes_cascade.detectMultiScale(cropImage, eyeScale, eyesMinNeighborgs)
          print('Can detect(eyes): ', len(eyesDetection))

          """if len(eyesDetection) == 2:
            for (ex, ey, ew, eh) in eyesDetection:
                cv2.rectangle(img, (x+ex, y+ey), (x+ex+ew, y+ey+eh), 
                              (255, 255, 255), 1)"""
            
          # 4. Cropping and scaling
          cropImage = img[y:y+h, x:x+w]
          cropImage = image_resize(cropImage, 32, 32)

          # 5. Normalize image 
          norm_img = np.zeros((300, 300))
          norm_img = cv2.normalize(cropImage, norm_img, 0, 255, cv2.NORM_MINMAX)

          # 6. Save image
          #ShowImage(norm_img)
          SaveImage(norm_img, len(faceDetection), index)

          # 7. Track metrics
          correct = correct + 1
          print("Correctly process detection!")
             
        if len(faceDetection) == 0 or len(faceDetection) > 1:
            # Save incorrect image and track metrics
            incorrect = incorrect + 1
            SaveImage(img, len(faceDetection), index)
            print("Incorrectly process detection!") 

    except Exception as e:
        fail = fail + 1
        print('Exception:', e)

tiempo_final = time() 
tiempo_ejecucion = tiempo_final - tiempo_inicial

metrics = {
            "CorrectProcess": correct,
            "IncorrectProcess": incorrect,
            "Accuracy": (correct / (correct + incorrect)) * 100,
            "Exceptions": fail,
            "ExecutionTime": tiempo_ejecucion     
          }
          
print('')
print(json.dumps(metrics))

x = json.dumps(metrics)
y = x.encode('utf-8')
with open('metrics_' + str(min_image) + 'to' + str(max_image) + '.json', 'w') as f:
  f.write(json.dumps(metrics))

files.download('metrics_' + str(min_image) + 'to' + str(max_image) + '.json')